In [1]:
from glob import glob
from tqdm import tqdm
import os, sys, pickle
import torch, librosa
from pyannote.audio import Pipeline
from config import config

/home/susmitha/anaconda3/envs/pyannote/lib/python3.10/site-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/home/susmitha/anaconda3/envs/pyannote/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/susmitha/anaconda3/envs/pyannote/lib/python3.10/site-packages/torch_audiomentations/utils/io.py:27: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [2]:
from pyannote.audio import Pipeline
pipeline = Pipeline.from_pretrained(config['pyannote']['vad_model'], use_auth_token=config['pyannote']['auth_key'])


Lightning automatically upgraded your loaded checkpoint from v1.1.3 to v2.1.4. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../../home/susmitha/.cache/torch/pyannote/models--pyannote--segmentation/snapshots/059e96f964841d40f1a5e755bb7223f76666bba4/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.7.1, yours is 2.2.0+cu118. Bad things might happen unless you revert torch to 1.x.


In [3]:
initial_params = {"onset": 0.6, "offset": 0.4, 
                  "min_duration_on": 0.0, "min_duration_off": 0.0}
pipeline.instantiate(initial_params)

In [4]:
device = torch.device(config['cude_device'])
pipeline.to(device)

In [5]:
podcasts = glob(os.path.join(config['podcast']['path'], "*.wav"))
finished_podcasts = glob(os.path.join(config['pyannote']['vad_output_path'], "*"))


podcasts = {os.path.basename(x).split('.')[0]:x for x in podcasts}
finished_podcasts = {os.path.basename(x):x for x in finished_podcasts}

remaining = list(set(podcasts.keys()) - finished_podcasts.keys())
remaining.sort()
podcasts = [podcasts[x] for x in remaining]

print('finished asr: {}\nremaining podcasts: {}'.format(len(finished_podcasts), len(podcasts)))

finished asr: 6787
remaining podcasts: 3


In [6]:
for pod in tqdm(podcasts):
    try:
        diarization = pipeline(pod)
    except:
        print("unable to process: ", pod)
        continue
    with open(config['pyannote']['vad_output_path'] + os.path.basename(pod).split('.')[0], 'wb') as f:
        pickle.dump(diarization, f)
        

  0%|                                                     | 0/3 [00:00<?, ?it/s]Note: Illegal Audio-MPEG-Header 0x1b88b821 at offset 32823.
Note: Trying to resync...
Note: Skipped 1024 bytes in input.
[src/libmpg123/parse.c:wetwork():1365] error: Giving up resync after 1024 bytes - your stream is not nice... (maybe increasing resync limit could help).
100%|█████████████████████████████████████████████| 3/3 [00:00<00:00, 66.23it/s]

unable to process:  /media/susmitha/HDD_14TB/NaturalVoices/podcasts/MSP-PODCAST_1185.wav
unable to process:  /media/susmitha/HDD_14TB/NaturalVoices/podcasts/MSP-PODCAST_1187.wav
unable to process:  /media/susmitha/HDD_14TB/NaturalVoices/podcasts/MSP-PODCAST_1516.wav
